### Xbox Controller Launcher

In [1]:
import os, time, pygame
import mujoco
import mujoco.viewer as vi
import numpy as np

os.environ["SDL_JOYSTICK_ALLOW_BACKGROUND_EVENTS"] = "1"

pygame.init()
# Set the width and height of the screen (width, height).
screen = pygame.display.set_mode((200, 200))
pygame.display.set_caption("holder")
# Loop until the user clicks the close button.
done = False
# Used to manage how fast the screen updates.
clock = pygame.time.Clock()
# Initialize the joysticks.
pygame.joystick.init()
joystick = pygame.joystick.Joystick(0)
joystick.init()


# UNCOMMENT LINE ACCORDINGLY
model = mujoco.MjModel.from_xml_path("./robotic-arm-scenes/kin-kin-scene.xml")
# model = mujoco.MjModel.from_xml_path("./robotic-arm-scenes/kin-ufactory-scene.xml")
# model = mujoco.MjModel.from_xml_path("./robotic-arm-scenes/ufactory-ufactory-scene.xml")
data = mujoco.MjData(model)

viewer = vi.launch_passive(model, data)
cam = viewer.cam
cam.azimuth = 89.25 ; cam.elevation = -67.5 ; cam.distance = 10.275441803128212
cam.lookat = np.array([ -0.00882946462815496 , 0.29350913433857984 , -0.21468602029580156 ])
done = False

motion_scaling = 0.03
def deadzoning(x):
    if(x < 0.05 and x > -0.05):
        return 0.0
    else:
        return x

while not done:
    mujoco.mj_step(model, data)
    viewer.sync()

    for event in pygame.event.get(): # User did something.
        if event.type == pygame.QUIT: # If user clicked close.
            done = True # Flag that we are done so we exit this loop.
        elif event.type == pygame.JOYBUTTONDOWN:
            if(event.button == 0):
                data.ctrl[0] = 0.0
                data.ctrl[1] = 0.0
                print("Reset!")
        elif event.type == pygame.JOYBUTTONUP:
            print("Button released")

    
    if(len(data.ctrl) == 2):
        LJS_horizontal_axis = joystick.get_axis(0)
        data.ctrl[0] = np.clip(data.ctrl[0] + motion_scaling*deadzoning(LJS_horizontal_axis), -3.0, 3.0)
        LJS_vertical_axis = joystick.get_axis(1)
        data.ctrl[1] = np.clip(data.ctrl[1] - motion_scaling*deadzoning(LJS_vertical_axis), -3.0, 3.0)
        Ltrigger_axis = joystick.get_axis(2)
    elif(len(data.ctrl) == 4):
        LJS_horizontal_axis = joystick.get_axis(0)
        data.ctrl[0] = np.clip(data.ctrl[0] + motion_scaling*deadzoning(LJS_horizontal_axis), -3.0, 3.0)
        LJS_vertical_axis = joystick.get_axis(1)
        data.ctrl[1] = np.clip(data.ctrl[1] - motion_scaling*deadzoning(LJS_vertical_axis), -3.0, 3.0)
        Ltrigger_axis = joystick.get_axis(2)
        
        RJS_horizontal_axis = joystick.get_axis(3)
        data.ctrl[2] = np.clip(data.ctrl[2] + motion_scaling*deadzoning(RJS_horizontal_axis), -3.0, 3.0)
        RJS_vertical_axis = joystick.get_axis(4)
        data.ctrl[3] = np.clip(data.ctrl[3] - motion_scaling*deadzoning(RJS_vertical_axis), -3.0, 3.0)
        Rtrigger_axis = joystick.get_axis(5)
    
    pygame.display.flip()
    # Limit to 10 frames per second.
    clock.tick(1000)
pygame.quit()

ModuleNotFoundError: No module named 'pygame'